In [1]:
import pandas as pd

In [34]:
df_email = pd.read_excel('C:/Users/Kimura/Desktop/Projetos/Projeto_01/Projeto_01/Bases_Dados/Emails.xlsx')
df_lojas = pd.read_csv('C:/Users/Kimura/Desktop/Projetos/Projeto_01/Projeto_01/Bases_Dados/Lojas.csv', encoding='latin1', sep=';')
df_vendas = pd.read_excel('C:/Users/Kimura/Desktop/Projetos/Projeto_01/Projeto_01/Bases_Dados/Vendas.xlsx')

In [42]:
df_email.head()

,Loja,Gerente,E-mail
0,Iguatemi Esplanada,Helena,pythonimpressionador+helena@gmail.com
1,Shopping Midway Mall,Alice,pythonimpressionador+alice@gmail.com
2,Norte Shopping,Laura,pythonimpressionador+laura@gmail.com
3,Shopping Iguatemi Fortaleza,Manuela,pythonimpressionador+manuela@gmail.com
4,Shopping União de Osasco,Valentina,pythonimpressionador+valentina@gmail.com


In [44]:
df_lojas.head()

,ID Loja,Loja
0,1,Iguatemi Esplanada
1,2,Shopping Midway Mall
2,3,Norte Shopping
3,4,Shopping Iguatemi Fortaleza
4,5,Shopping União de Osasco


In [46]:
df_vendas.head()

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,1,Sapato Estampa,1,358,358
1,1,2019-01-01,1,Camiseta,2,180,360
2,1,2019-01-01,1,Sapato Xadrez,1,368,368
3,2,2019-01-02,3,Relógio,3,200,600
4,2,2019-01-02,3,Chinelo Liso,1,71,71
